After analysing the mail's thoroughly in the eda section and analysing their importances visually ,it's now time to apply statistical techqniues to back our ideas.

After doing EDA we have decided to stick on to the following features in my table, we will still apply statistical meausres on all these feartures

1. mail
2. word_length
3. clean_text
5. dollar_count
6. exclamation_count
7. diacritic_count
8. has_email
9. dominant topic
10. type


## Libraries

In [457]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from textblob import TextBlob
import string
import seaborn as sns
from collections import Counter
import bs4
import spacy
import re
from bs4 import BeautifulSoup
import unicodedata
import matplotlib.pyplot as plt
import joblib

In [458]:
import re
import unicodedata
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from nltk.corpus import stopwords
from sklearn.preprocessing import FunctionTransformer

In [459]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
    

In [460]:
df=pd.read_csv("../data/train.csv")

## Replace Null values by empty string 

In [461]:
df['mail'] = df['mail'].fillna('')

In [462]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14920 entries, 0 to 14919
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   mail    14920 non-null  object
 1   type    14920 non-null  object
dtypes: object(2)
memory usage: 233.3+ KB


Now we will add all those columns to the dataset into the df 

In [463]:
df['word_length'] = df['mail'].apply(lambda x: len(str(x).split()))

In [464]:
df

,mail,type,word_length
0,"On Sun, 11 Aug 2002 bitbitch@magnesium.net wro...",Safe Email,111
1,hello all : please send an email to : ibuyit @...,Safe Email,531
2,"idioms content - length : 99 please send "" app...",Safe Email,26
3,re : work from home this is one time message ....,Phishing Email,52
4,test results - resend by december 14 th the su...,Phishing Email,190
...,...,...,...
14915,ask for it bait - excelled @ em . ca - thousan...,Phishing Email,90
14916,a computer and internet connection for you and...,Safe Email,533
14917,first israeli seminar on computational linguis...,Safe Email,342
14918,enrononline weekly public report for november ...,Safe Email,312


In [465]:
df['dollar_count'] = df['mail'].str.count('$')
df

,mail,type,word_length,dollar_count
0,"On Sun, 11 Aug 2002 bitbitch@magnesium.net wro...",Safe Email,111,2
1,hello all : please send an email to : ibuyit @...,Safe Email,531,1
2,"idioms content - length : 99 please send "" app...",Safe Email,26,1
3,re : work from home this is one time message ....,Phishing Email,52,1
4,test results - resend by december 14 th the su...,Phishing Email,190,1
...,...,...,...,...
14915,ask for it bait - excelled @ em . ca - thousan...,Phishing Email,90,1
14916,a computer and internet connection for you and...,Safe Email,533,1
14917,first israeli seminar on computational linguis...,Safe Email,342,1
14918,enrononline weekly public report for november ...,Safe Email,312,1


In [466]:
df['exclamation_count'] = df['mail'].str.count('!')
df.sample(5)

,mail,type,word_length,dollar_count,exclamation_count
6181,technology & foreign language teaching transfo...,Safe Email,304,1,0
4532,empty,Safe Email,1,1,0
11691,Is Your Credit A Mess?\nDo you want Perfect Cr...,Phishing Email,114,2,7
2360,re : gulf coast pipeline company dyersdale are...,Safe Email,644,1,0
3279,job at max planck institute for psycholinguist...,Safe Email,261,1,0


In [467]:
df['has_email'] = df['mail'].str.contains(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}', regex=True)
df.sample(5)

,mail,type,word_length,dollar_count,exclamation_count,has_email
13931,5 % guaranteed for eight years 5 % guaranteed ...,Phishing Email,147,1,1,False
10708,Gordon Mohr:\n>It was clear you were talking a...,Safe Email,196,2,0,False
12398,student organisation of linguistics in europe ...,Safe Email,1448,1,4,False
1033,nystesol applied ling second call for proposal...,Safe Email,689,1,1,False
4225,give your pc a tune - up with system mechanic ...,Phishing Email,270,1,2,False


In [468]:
def clean_text(text):

    # Import stop words
    # Convert text to lowercase and split into words
    words = text.lower().split()
    
    # Remove stop words
    words = [word for word in words if word not in stop_words]
    
    # Join words back together
    text = ' '.join(words)
    
    # Convert to string
    text = str(text)
    # Remove punctuation except ! and $
    text = re.sub(r'[^\w\s!$]', '', text)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove email addresses
    text = re.sub(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}', '', text)
    
    # Remove phone numbers
    text = re.sub(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', '', text)
    
    return text

df['clean_text'] = df['mail'].apply(clean_text)
df.sample(5)

,mail,type,word_length,dollar_count,exclamation_count,has_email,clean_text
3383,re : panenergy marketing march 2000 production...,Safe Email,407,1,0,False,panenergy marketing march 2000 production dar...
8290,hotlist update for bhlp and upstream products ...,Safe Email,1212,1,0,False,hotlist update bhlp upstream products louise ...
1149,professional logo for you now working on your ...,Phishing Email,206,1,2,False,professional logo working company image star...
6857,schedule crawler : hourahead failure start dat...,Safe Email,28,1,0,False,schedule crawler hourahead failure start date...
6112,re : tgp sabine - - - - - - - - - - - - - - - ...,Safe Email,485,1,0,False,tgp sabine forwarded ed...


In [469]:
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
text_converter = tfidf.fit(df['clean_text'])
text_matrix=tfidf.transform(df['clean_text'])
# Apply NMF
nmf_model = NMF(n_components=10, random_state=42)
nmf_matrix = nmf_model.fit(text_matrix)
nmf_output=nmf_model.transform(text_matrix)

# Get dominant topic
df['dominant_topic'] = nmf_output.argmax(axis=1)

In [470]:
df.sample(5)

,mail,type,word_length,dollar_count,exclamation_count,has_email,clean_text,dominant_topic
10978,"URL: http://www.newsisfree.com/click/-4,872399...",Safe Email,11,2,0,False,url date 20021010t0326540100arts curators des...,2
3515,honest family oriented home based business wit...,Phishing Email,1072,1,11,False,honest family oriented home based business res...,7
7811,lim software upgrade notice all lim mimic & ex...,Safe Email,136,1,0,False,lim software upgrade notice lim mimic excel a...,6
8767,= ? iso - 8859 - 7 ? q ? = 5 b = 3 f = 5 d _ f...,Phishing Email,229,1,1,False,iso 8859 7 q 5 b 3 f 5 _ fwd _ got _...,8
4159,"re : may wellhead "" spot "" purchases - request...",Safe Email,504,1,0,False,may wellhead spot purchases request vance ...,1


In [471]:
def count_diacritics(text):
    return sum(1 for char in text if unicodedata.combining(char) or 
              unicodedata.category(char).startswith('L') and unicodedata.decomposition(char))

df['diacritic_count'] = df['clean_text'].apply(count_diacritics)


In [472]:
df['diacritic_count'].value_counts()

diacritic_count
0       13998
1         199
2         113
3          63
4          54
        ...  
33          1
181         1
3455        1
379         1
45          1
Name: count, Length: 111, dtype: int64

In [473]:
# Save the processed dataframe to CSV
df.to_csv('../data/after_eda_train.csv', index=False)

In [474]:
df=pd.read_csv("../Data/after_eda_train.csv")

In [475]:
df

,mail,type,word_length,dollar_count,exclamation_count,has_email,clean_text,dominant_topic,diacritic_count
0,"On Sun, 11 Aug 2002 bitbitch@magnesium.net wro...",Safe Email,111,2,0,True,sun 11 aug 2002 bitbitchmagnesiumnet wrote mea...,3,0
1,hello all : please send an email to : ibuyit @...,Safe Email,531,1,4,False,hello please send email ibuyit enron com s...,6,0
2,"idioms content - length : 99 please send "" app...",Safe Email,26,1,1,False,idioms content length 99 please send appala...,4,0
3,re : work from home this is one time message ....,Phishing Email,52,1,0,False,work home one time message received email ex...,7,0
4,test results - resend by december 14 th the su...,Phishing Email,190,1,1,False,test results resend december 14 th super pain...,5,0
...,...,...,...,...,...,...,...,...,...
14915,ask for it bait - excelled @ em . ca - thousan...,Phishing Email,90,1,0,False,ask bait excelled em ca thousands availabl...,7,0
14916,a computer and internet connection for you and...,Safe Email,533,1,0,False,computer internet connection family know tech...,6,0
14917,first israeli seminar on computational linguis...,Safe Email,342,1,0,False,first israeli seminar computational linguistic...,4,0
14918,enrononline weekly public report for november ...,Safe Email,312,1,0,False,enrononline weekly public report november 16 ...,9,0


In [476]:
df.describe()

,word_length,dollar_count,exclamation_count,dominant_topic,diacritic_count
count,1.492000e+04,14920.000000,14920.000000,14920.000000,14920.000000
mean,5.800428e+02,1.194035,2.657909,4.424665,2.407507
std,2.888706e+04,0.395469,117.830133,2.825621,46.353370
min,0.000000e+00,1.000000,0.000000,0.000000,0.000000
25%,7.400000e+01,1.000000,0.000000,3.000000,0.000000
50%,1.590000e+02,1.000000,0.000000,4.000000,0.000000
75%,3.502500e+02,1.000000,1.000000,7.000000,0.000000
max,3.527576e+06,2.000000,14368.000000,9.000000,3455.000000


In [477]:
(df['word_length']>1500).sum()

np.int64(513)

In [478]:
df.isna().sum()

mail                 11
type                  0
word_length           0
dollar_count          0
exclamation_count     0
has_email             0
clean_text           15
dominant_topic        0
diacritic_count       0
dtype: int64

# replace null value by empty string

In [479]:
df['mail'] = df['mail'].fillna('')
df['clean_text'] = df['clean_text'].fillna('')

In [480]:
df.isna().sum()

mail                 0
type                 0
word_length          0
dollar_count         0
exclamation_count    0
has_email            0
clean_text           0
dominant_topic       0
diacritic_count      0
dtype: int64

We will remove all the emails with mails with mail length exceeding 2000 to get rid of unncessary outliers

In [481]:
df=df[df["word_length"]<2000]

In [482]:
df

,mail,type,word_length,dollar_count,exclamation_count,has_email,clean_text,dominant_topic,diacritic_count
0,"On Sun, 11 Aug 2002 bitbitch@magnesium.net wro...",Safe Email,111,2,0,True,sun 11 aug 2002 bitbitchmagnesiumnet wrote mea...,3,0
1,hello all : please send an email to : ibuyit @...,Safe Email,531,1,4,False,hello please send email ibuyit enron com s...,6,0
2,"idioms content - length : 99 please send "" app...",Safe Email,26,1,1,False,idioms content length 99 please send appala...,4,0
3,re : work from home this is one time message ....,Phishing Email,52,1,0,False,work home one time message received email ex...,7,0
4,test results - resend by december 14 th the su...,Phishing Email,190,1,1,False,test results resend december 14 th super pain...,5,0
...,...,...,...,...,...,...,...,...,...
14915,ask for it bait - excelled @ em . ca - thousan...,Phishing Email,90,1,0,False,ask bait excelled em ca thousands availabl...,7,0
14916,a computer and internet connection for you and...,Safe Email,533,1,0,False,computer internet connection family know tech...,6,0
14917,first israeli seminar on computational linguis...,Safe Email,342,1,0,False,first israeli seminar computational linguistic...,4,0
14918,enrononline weekly public report for november ...,Safe Email,312,1,0,False,enrononline weekly public report november 16 ...,9,0


In [483]:
df.describe()

,word_length,dollar_count,exclamation_count,dominant_topic,diacritic_count
count,14610.000000,14610.000000,14610.000000,14610.000000,14610.000000
mean,267.570910,1.195346,1.363860,4.405339,2.044353
std,318.233848,0.396480,4.104309,2.822002,36.840543
min,0.000000,1.000000,0.000000,0.000000,0.000000
25%,73.000000,1.000000,0.000000,3.000000,0.000000
50%,155.000000,1.000000,0.000000,4.000000,0.000000
75%,329.000000,1.000000,1.000000,7.000000,0.000000
max,1998.000000,2.000000,179.000000,9.000000,1900.000000


## type vs word_length

In [484]:
from scipy import stats

# Split data into two groups based on email type
safe_emails = df[df['type'] == 'Safe Email']['word_length']
phishing_emails = df[df['type'] == 'Phishing Email']['word_length']

# Perform independent t-test
t_stat, p_value = stats.ttest_ind(safe_emails, phishing_emails)

print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("\nHypothesis: There is a statistically significant difference in word length between safe and phishing emails")
else:
    print("\nHypothesis: There is no statistically significant difference in word length between safe and phishing emails")


t-statistic: 8.2076
p-value: 0.0000

Hypothesis: There is a statistically significant difference in word length between safe and phishing emails


 let us see between type and dollar_count

In [485]:
# Split data into two groups based on email type
safe_emails = df[df['type'] == 'Safe Email']['dollar_count']
phishing_emails = df[df['type'] == 'Phishing Email']['dollar_count']

# Perform independent t-test
t_stat, p_value = stats.ttest_ind(safe_emails, phishing_emails)

print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("\nHypothesis: There is a statistically significant difference in dollar count between safe and phishing emails")
else:
    print("\nHypothesis: There is no statistically significant difference in dollar count between safe and phishing emails")


t-statistic: 12.4348
p-value: 0.0000

Hypothesis: There is a statistically significant difference in dollar count between safe and phishing emails


type vs exclamation count

In [486]:
# Split data into two groups based on email type
safe_emails = df[df['type'] == 'Safe Email']['exclamation_count']
phishing_emails = df[df['type'] == 'Phishing Email']['exclamation_count']

# Perform independent t-test
t_stat, p_value = stats.ttest_ind(safe_emails, phishing_emails)

print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("\nHypothesis: There is a statistically significant difference in excalamtion count between safe and phishing emails")
else:
    print("\nHypothesis: There is no statistically significant difference in exclamation count between safe and phishing emails")


t-statistic: -29.9488
p-value: 0.0000

Hypothesis: There is a statistically significant difference in excalamtion count between safe and phishing emails


diacritic_count vs type

In [487]:
# Split data into two groups based on email type
safe_emails = df[df['type'] == 'Safe Email']['diacritic_count']
phishing_emails = df[df['type'] == 'Phishing Email']['diacritic_count']

# Perform independent t-test
t_stat, p_value = stats.ttest_ind(safe_emails, phishing_emails)

print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("\nHypothesis: There is a statistically significant difference in diacritic count between safe and phishing emails")
else:
    print("\nHypothesis: There is no statistically significant difference in diacritic count between safe and phishing emails")


t-statistic: -6.6863
p-value: 0.0000

Hypothesis: There is a statistically significant difference in diacritic count between safe and phishing emails


 now we will check for has email vs type , we will use chi square test

In [488]:
contingency_table = pd.crosstab(df['has_email'], df['type'])

chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("\nHypothesis: There is a statistically significant association between has_email and type")
else:
    print("\nHypothesis: There is no statistically significant association between has_email and type")


Chi-square statistic: 235.1181
p-value: 0.0000

Hypothesis: There is a statistically significant association between has_email and type


In [489]:
# Add a column named has_phone_no
df['has_phone_no'] = df['mail'].str.contains(r'\d{3}[-.\s]?\d{3}[-.\s]?\d{4}', na=False)

# Create a contingency table for has_phone_no and type
contingency_table_phone = pd.crosstab(df['has_phone_no'], df['type'])

# Perform chi-square test
chi2_stat_phone, p_value_phone, dof_phone, expected_phone = stats.chi2_contingency(contingency_table_phone)

print(f"Chi-square statistic for phone number: {chi2_stat_phone:.4f}")
print(f"p-value for phone number: {p_value_phone:.4f}")

# Interpret results
alpha = 0.05
if p_value_phone < alpha:
    print("\nHypothesis: There is a statistically significant association between has_phone_no and type")
else:
    print("\nHypothesis: There is no statistically significant association between has_phone_no and type")


Chi-square statistic for phone number: 6.1489
p-value for phone number: 0.0131

Hypothesis: There is a statistically significant association between has_phone_no and type


/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_38811/3874176793.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['has_phone_no'] = df['mail'].str.contains(r'\d{3}[-.\s]?\d{3}[-.\s]?\d{4}', na=False)


# just to check normality of my numeric features

In [490]:
from scipy.stats import anderson
columns_to_check = ['dollar_count', 'diacritic_count', 'exclamation_count', 'word_length']
for column in columns_to_check:
    result = anderson(df[column].dropna(), dist='norm')
    print(f"Results for {column}:")
    print(f"Statistic: {result.statistic:.4f}")
    print(f"Critical Values: {result.critical_values}")
    print(f"Significance Level: {result.significance_level}")
    
    # Check if the statistic is less than the critical value for the 5% significance level
    if result.statistic < result.critical_values[2]:  # 2 corresponds to 5% significance level
        print(f"{column} is normally distributed.\n")
    else:
        print(f"{column} is not normally distributed.\n")


Results for dollar_count:
Statistic: 3930.7934
Critical Values: [0.576 0.656 0.787 0.918 1.092]
Significance Level: [15.  10.   5.   2.5  1. ]
dollar_count is not normally distributed.

Results for diacritic_count:
Statistic: 5323.0481
Critical Values: [0.576 0.656 0.787 0.918 1.092]
Significance Level: [15.  10.   5.   2.5  1. ]
diacritic_count is not normally distributed.

Results for exclamation_count:
Statistic: 2899.6118
Critical Values: [0.576 0.656 0.787 0.918 1.092]
Significance Level: [15.  10.   5.   2.5  1. ]
exclamation_count is not normally distributed.

Results for word_length:
Statistic: 1213.6301
Critical Values: [0.576 0.656 0.787 0.918 1.092]
Significance Level: [15.  10.   5.   2.5  1. ]
word_length is not normally distributed.



since we are not intrested in applying linear models it does not really matter wheather they are nroammly distrbiuted or not

In [491]:
# Count phishing and non-phishing emails for has_phone_no
phone_counts = df.groupby(['has_phone_no', 'type']).size().unstack(fill_value=0)
print("Counts for has_phone_no:")
print(phone_counts)

# Count phishing and non-phishing emails for has_email
email_counts = df.groupby(['has_email', 'type']).size().unstack(fill_value=0)
print("\nCounts for has_email:")
print(email_counts)


Counts for has_phone_no:
type          Phishing Email  Safe Email
has_phone_no                            
False                   5433        8276
True                     319         582

Counts for has_email:
type       Phishing Email  Safe Email
has_email                            
False                5251        7282
True                  501        1576


In [492]:
numeric_colums=["diacritic_count","dollar_count","exclamation_count","word_length"]
categorical_colums=["has_email","has_phone_no"]

In [493]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif

# Assuming 'df' is your DataFrame and 'type' is the target variable
# Convert the target variable to numeric (0 for safe, 1 for phishing)
df['type'] = df['type'].map({'Safe Email': 0, 'Phishing Email': 1})  # Adjust mapping as needed

# List of numeric columns
numeric_columns = ["diacritic_count", "dollar_count", "exclamation_count", "word_length"]

# Calculate MI scores
mi_scores = mutual_info_classif(df[numeric_columns], df['type'])

# Create a DataFrame to display the results
mi_results = pd.DataFrame({'Feature': numeric_columns, 'MI Score': mi_scores})

# Print the MI scores
print(mi_results)

/var/folders/7w/qs77cz_11630n9gydm61vjpc0000gn/T/ipykernel_38811/3168255693.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['type'] = df['type'].map({'Safe Email': 0, 'Phishing Email': 1})  # Adjust mapping as needed


             Feature  MI Score
0    diacritic_count  0.012935
1       dollar_count  0.007697
2  exclamation_count  0.090692
3        word_length  0.018908


In [494]:

# Calculate MI scores
mi_scores = mutual_info_classif(df[categorical_colums], df['type'])

# Create a DataFrame to display the results
mi_results = pd.DataFrame({'Feature': categorical_colums, 'MI Score': mi_scores})

# Print the MI scores
print(mi_results)

        Feature  MI Score
0     has_email  0.005317
1  has_phone_no  0.000000


| Feature            | Statistically Significant | MI Score |
|--------------------|--------------------------|----------|
| diacritic_count    | Yes                      | 0.017 |
| dollar_count       | Yes                      | 0. 025|
| exclamation_count   | Yes                      | 0.09 |
| word_length        | Yes                      | 0.022 |
| has_email          | Yes                      | 0.006 |
| has_phone_no      | Yes                      | 0.000 |


In [495]:
categorical_colums

['has_email', 'has_phone_no']

## Now coming to what I feel is trump card, dominant topic

In [496]:
df['dominant_topic'].value_counts()

dominant_topic
0    2414
6    2149
7    1993
3    1968
4    1878
9    1387
5    1159
1     591
2     556
8     515
Name: count, dtype: int64

In [497]:
from sklearn.metrics import mutual_info_score

# Calculate the mutual information score between 'dominant_topic' and 'type'
mutual_info = mutual_info_score(df['dominant_topic'], df['type'])

# Print the mutual information score
print(f'Mutual Information Score between dominant_topic and type: {mutual_info}')


Mutual Information Score between dominant_topic and type: 0.3430812529564477


as expected the mutual information is the highest for this.

so after analysing all the features we have decided to stay with the folliowing features -:
1. dominant_topic
2. dollar_count
3. exclamation_count
4. diacritic_count

In [498]:
df=df[['mail','type','clean_text','dominant_topic','dollar_count','exclamation_count','diacritic_count']]

In [499]:
df

,mail,type,clean_text,dominant_topic,dollar_count,exclamation_count,diacritic_count
0,"On Sun, 11 Aug 2002 bitbitch@magnesium.net wro...",0,sun 11 aug 2002 bitbitchmagnesiumnet wrote mea...,3,2,0,0
1,hello all : please send an email to : ibuyit @...,0,hello please send email ibuyit enron com s...,6,1,4,0
2,"idioms content - length : 99 please send "" app...",0,idioms content length 99 please send appala...,4,1,1,0
3,re : work from home this is one time message ....,1,work home one time message received email ex...,7,1,0,0
4,test results - resend by december 14 th the su...,1,test results resend december 14 th super pain...,5,1,1,0
...,...,...,...,...,...,...,...
14915,ask for it bait - excelled @ em . ca - thousan...,1,ask bait excelled em ca thousands availabl...,7,1,0,0
14916,a computer and internet connection for you and...,0,computer internet connection family know tech...,6,1,0,0
14917,first israeli seminar on computational linguis...,0,first israeli seminar computational linguistic...,4,1,0,0
14918,enrononline weekly public report for november ...,0,enrononline weekly public report november 16 ...,9,1,0,0


In [500]:
df

,mail,type,clean_text,dominant_topic,dollar_count,exclamation_count,diacritic_count
0,"On Sun, 11 Aug 2002 bitbitch@magnesium.net wro...",0,sun 11 aug 2002 bitbitchmagnesiumnet wrote mea...,3,2,0,0
1,hello all : please send an email to : ibuyit @...,0,hello please send email ibuyit enron com s...,6,1,4,0
2,"idioms content - length : 99 please send "" app...",0,idioms content length 99 please send appala...,4,1,1,0
3,re : work from home this is one time message ....,1,work home one time message received email ex...,7,1,0,0
4,test results - resend by december 14 th the su...,1,test results resend december 14 th super pain...,5,1,1,0
...,...,...,...,...,...,...,...
14915,ask for it bait - excelled @ em . ca - thousan...,1,ask bait excelled em ca thousands availabl...,7,1,0,0
14916,a computer and internet connection for you and...,0,computer internet connection family know tech...,6,1,0,0
14917,first israeli seminar on computational linguis...,0,first israeli seminar computational linguistic...,4,1,0,0
14918,enrononline weekly public report for november ...,0,enrononline weekly public report november 16 ...,9,1,0,0


In [501]:
df.to_csv('../data/final_train.csv', index=False)

# Custom Transformers for Pipelines

In [406]:
class DollarCountTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([text.count('$') for text in X]).reshape(-1, 1)

In [407]:
class ExclamationCountTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([text.count('!') for text in X]).reshape(-1, 1)

In [408]:
class CleanTextTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [self.clean_text(text) for text in X]

    def clean_text(self, text):
        words = text.lower().split()
        words = [word for word in words if word not in stop_words]
        text = ' '.join(words)
        text = str(text)
        text = re.sub(r'[^\w\s!$]', '', text)
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}', '', text)
        text = re.sub(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', '', text)
        return text

In [409]:
class DiacriticCountTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([self.count_diacritics(text) for text in X]).reshape(-1, 1)

    def count_diacritics(self, text):
        return sum(1 for char in text if unicodedata.combining(char) or 
                  unicodedata.category(char).startswith('L') and unicodedata.decomposition(char))

In [410]:
class NMFTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, n_components=10, random_state=42):
        self.n_components = n_components
        self.random_state = random_state
        self.nmf = NMF(n_components=n_components, random_state=random_state)
        self.tfidf = TfidfVectorizer(max_features=1000, stop_words='english')

    def fit(self, X, y=None):
        text_matrix = self.tfidf.fit_transform(X)
        self.nmf.fit(text_matrix)
        return self

    def transform(self, X):
        text_matrix = self.tfidf.transform(X)
        nmf_output = self.nmf.transform(text_matrix)
        return nmf_output.argmax(axis=1).reshape(-1, 1)

In [454]:
def clean_text(text):

    # Import stop words
    # Convert text to lowercase and split into words
    words = text.lower().split()
    
    # Remove stop words
    words = [word for word in words if word not in stop_words]
    
    # Join words back together
    text = ' '.join(words)
    
    # Convert to string
    text = str(text)
    # Remove punctuation except ! and $
    text = re.sub(r'[^\w\s!$]', '', text)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove email addresses
    text = re.sub(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}', '', text)
    
    # Remove phone numbers
    text = re.sub(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', '', text)
    
    return text

In [455]:
# Step 2: Custom Transformer to Add New Features
class FeatureAdder(BaseEstimator, TransformerMixin):
    """Custom Transformer to add new numeric features based on text analysis"""
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_transformed = X.copy()  # Preserve original DataFrame
        
        # Apply text cleaning and create 'clean_text' column
        X_transformed['clean_text'] = X_transformed['mail'].apply(clean_text)
        
        # Count occurrences of '$' and '!'
        X_transformed['dollar_count'] = X_transformed['mail'].str.count(r'\$')
        X_transformed['exclamation_count'] = X_transformed['mail'].str.count(r'!')
        
        # Count diacritics
        def count_diacritics(text):
            return sum(1 for char in text if unicodedata.combining(char) or 
                       (unicodedata.category(char).startswith('L') and 
                        unicodedata.decomposition(char)))
        
        X_transformed['diacritic_count'] = X_transformed['clean_text'].apply(count_diacritics)
        
        return X_transformed

In [456]:
class NMFTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, n_components=10, random_state=42):
        self.n_components = n_components
        self.random_state = random_state
        self.nmf = NMF(n_components=n_components, random_state=random_state)
        self.tfidf = TfidfVectorizer(max_features=1000, stop_words='english')

    def fit(self, X, y=None):
        text_matrix = self.tfidf.fit_transform(X)
        self.nmf.fit(text_matrix)
        return self

    def transform(self, X):
        text_matrix = self.tfidf.transform(X)
        nmf_output = self.nmf.transform(text_matrix)
        return nmf_output.argmax(axis=1).reshape(-1, 1)

In [ ]:
# Step 5: Create Column Transformer
column_transformer = ColumnTransformer(
    transformers=[
        ('nmf', NMFTransformer(), 'clean_text'),  # Apply NMF on clean_text
        ('dollar', 'passthrough', ['dollar_count']),  # Keep dollar_count
        ('exclamation', 'passthrough', ['exclamation_count']),  # Keep exclamation_count
        ('clean_text', 'passthrough', ['clean_text']),
        ('diacritic', 'passthrough', ['diacritic_count']) # Keep diacritic_count
    ],
    remainder='passthrough'  # Keep other columns unchanged
)

# Step 6: Create Full Pipeline
pipeline = Pipeline([
    ('feature_adder', FeatureAdder()),  # Add new features and clean text
    ('feature_extraction', column_transformer)  # Apply NMF and keep extracted features
])


In [ ]:
df = pd.DataFrame({
    'mail': ["Win $1000 now!!! Click here", "Hello, let's meet tomorrow!", "Urgent! Your account is at risk!"],
    'type': ['Phishing Email', 'Safe Email', 'Phishing Email']
})

df_transformed = pipeline.fit_transform(df)
df_transformed
# Convert back to DataFrame with proper column names

In [443]:
df_transformed = pd.DataFrame(df_transformed, columns=[
    'dominant_topic','dollar_count', 'exclamation_count', 'diacritic_count', 'mail', 'type', 'clean_text'
])

df_transformed

,dominant_topic,dollar_count,exclamation_count,diacritic_count,mail,type,clean_text
0,4,1,3,win $1000 now!!! click,0,Win $1000 now!!! Click here,Phishing Email
1,1,0,1,hello lets meet tomorrow!,0,"Hello, let's meet tomorrow!",Safe Email
2,3,0,2,urgent! account risk!,0,Urgent! Your account is at risk!,Phishing Email


In [446]:
df1 = pd.DataFrame({
    'mail': ["wowww comeonnn superrr", "lord is great", "money money money$$$$"],
    'type': ['Phishing Email', 'Safe Email', 'Phishing Email']
})
df1_transformed = pipeline.transform(df1)

# Convert back to DataFrame with proper column names
df1_transformed = pd.DataFrame(df1_transformed, columns=[
    'dominant_topic','dollar_count', 'exclamation_count', 'diacritic_count', 'mail', 'type', 'clean_text'
])

In [447]:
df1_transformed

,dominant_topic,dollar_count,exclamation_count,diacritic_count,mail,type,clean_text
0,0,0,0,wowww comeonnn superrr,0,wowww comeonnn superrr,Phishing Email
1,0,0,0,lord great,0,lord is great,Safe Email
2,0,4,0,money money money$$$$,0,money money money$$$$,Phishing Email


In [448]:
joblib.dump(pipeline, "email_pipeline.pkl")
print("Pipeline saved successfully!")

Pipeline saved successfully!
